In [ ]:
import numpy as np
import mxnet as mx
from collections import namedtuple, Counter
from unidecode import unidecode
from itertools import groupby
from mxnet.io import DataBatch, DataIter
from random import shuffle
from mxnet import ndarray

# import deepdish as dd

import operator
import pickle
import re
import warnings

In [2]:
# Get rid of annoying Python deprecation warnings from built-in JSON encoder
warnings.filterwarnings("ignore", category=DeprecationWarning)   

In [3]:
# Decode text as UTF-8
# Remove diacritical signs and convert to Latin alphabet
# Separate punctuation as separate "words"
def tokenize_text(fname, vocab=None, invalid_label=0, start_label=1, sep_punctuation=True):
    lines = unidecode(open(fname).read().decode('utf-8')).split('\n')
#     lines = [x for x in lines if x]
    lines = map(lambda x: re.findall(r"\w+|[^\w\s]", x, re.UNICODE), lines)    
    sentences, vocab = mx.rnn.encode_sentences(lines, vocab=vocab, invalid_label=invalid_label, start_label=start_label)
    return sentences, vocab

Dataset = namedtuple(
    'Dataset', 
    ['src_sent', 'src_vocab', 'inv_src_vocab', 'targ_sent', 'targ_vocab', 'inv_targ_vocab'])

def invert_dict(d):
    return {v: k for k, v in d.iteritems()}


def get_data(src_path, targ_path, start_label=1, invalid_label=0, pad_symbol='<PAD>'):
    src_sent, src_vocab = tokenize_text(src_path, start_label=start_label,
                                invalid_label=invalid_label)
    
    src_vocab[pad_symbol] = invalid_label
    inv_src_vocab = invert_dict(src_vocab)

    targ_sent, targ_vocab = tokenize_text(targ_path, start_label=start_label, #new_start+1,
                                          invalid_label=invalid_label)
    
    targ_vocab[pad_symbol] = invalid_label
    inv_targ_vocab = invert_dict(targ_vocab)
    
    return Dataset(
        src_sent=src_sent, src_vocab=src_vocab, inv_src_vocab=inv_src_vocab,
        targ_sent=targ_sent, targ_vocab=targ_vocab, inv_targ_vocab=inv_targ_vocab)

In [4]:
def persist_dataset(dataset, path):
    with open(path, 'wb+') as fileobj:
        pickle.dump(dataset, fileobj)
        
def load_dataset(path):
    with open(path, 'rb') as fileobj:
        return pickle.load(fileobj)

In [6]:
dataset = \
    get_data(
        src_path='./data/europarl-v7.es-en.en_train_small',
        targ_path='./data/europarl-v7.es-en.es_train_small',
        start_label=1,
        invalid_label=0
    )

In [7]:
# TODO: Don't convert from NumPy to MxNet NDArray at the last minute.
# Use MxNet NDArray from the beginning. This will require some changes
# though, e.g. MxNet NDArray doesn't allow a len() call (use shape), etc.

class Seq2SeqIter(DataIter):

    class TwoDBisect:
        def __init__(self, buckets):
            self.buckets = sorted(buckets, key=operator.itemgetter(0, 1))
            self.x, self.y = zip(*buckets)
            self.x, self.y = np.array(list(self.x)), np.array(list(self.y))

        def twod_bisect(self, source, target):    
            offset1 = np.searchsorted(self.x, len(source), side='left')
            offset2 = np.where(self.y[offset1:] >= len(target))[0]        
            return self.buckets[offset1 + offset2[0]]     
    
    def __init__(
        self, dataset, buckets=None, batch_size=32, max_sent_len=None,
        data_name='data', label_name='softmax_label', dtype=np.int32, layout='NTC'):
        self.data_name = data_name
        self.label_name = label_name
        self.dtype = dtype
        self.layout = layout
        self.batch_size = batch_size
        self.src_sent = dataset.src_sent
        self.targ_sent = dataset.targ_sent
        if buckets:
            z = zip(*buckets)
            self.max_sent_len = max(max(z[0]), max(z[1]))
        else:
            self.max_sent_len = max_sent_len
        if self.max_sent_len:
            self.src_sent, self.targ_sent = self.filter_long_sent(
                self.src_sent, self.targ_sent, self.max_sent_len) 
        self.src_vocab = dataset.src_vocab
        self.targ_vocab = dataset.targ_vocab
        self.inv_src_vocab = dataset.inv_src_vocab
        self.inv_targ_vocab = dataset.inv_targ_vocab
        # Can't filter smaller counts per bucket if those sentences still exist!
        self.buckets = buckets if buckets else self.gen_buckets(
            self.src_sent, self.targ_sent, filter_smaller_counts_than=1, max_sent_len=max_sent_len)
        self.bisect = Seq2SeqIter.TwoDBisect(self.buckets)
        self.max_sent_len = max_sent_len
        self.pad_id = self.src_vocab['<PAD>']
        # After bucketization, we should probably del self.src_sent and self.targ_sent
        # to free up memory.
        self.bucketed_data, self.bucket_idx_to_key = self.bucketize()
        self.bucket_key_to_idx = invert_dict(dict(enumerate(self.bucket_idx_to_key)))
        self.interbucket_idx = -1
        self.curr_bucket_id = None
        self.curr_chunks = None
        self.curr_buck = None
        self.switch_bucket = True
        self.num_buckets = len(self.bucket_idx_to_key)
        self.bucket_iterator_indices = list(range(self.num_buckets))

    
    def bucketize(self):
        tuples = []
        ctr = 0
        for src, targ in zip(self.src_sent, self.targ_sent):
            len_tup = self.bisect.twod_bisect(src, targ)
            rev_src = src[::-1] 
            tuples.append((src, targ, len_tup))
            
        sorted_tuples = sorted(tuples, key=operator.itemgetter(2))
        grouped = groupby(sorted_tuples, lambda x: x[2])
        bucketed_data = [] 
        bucket_idx_to_key = []
        
        for group in grouped:
            
            # get src and targ sentences, ignore the last elem of the tuple 
            # (the grouping key of (src_len, targ_len))
            key, value = group[0], map(lambda x: x[:2], group[1])
            if len(value) < self.batch_size:
                continue

            # create padded representation
            new_src = np.full((len(value), key[0]), self.pad_id, dtype=self.dtype)
            new_targ = np.full((len(value), key[1]), self.pad_id, dtype=self.dtype)
            
            for idx, example in enumerate(value):
                curr_src, curr_targ = example
                rev_src = curr_src[::-1]
                new_src[idx, :-(len(rev_src)+1):-1] = curr_src
                new_targ[idx, :len(curr_targ)] = curr_targ
                            
            bucketed_data.append((new_src, new_targ))

            bucket_idx_to_key.append(key)
        return bucketed_data, bucket_idx_to_key
    
    def current_bucket_key(self):
        return self.bucket_idx_to_key[self.interbucket_idx]
    
    def current_bucket_index(self):
        return self.bucket_iterator_indices[self.interbucket_idx]

    # shuffle the data within buckets, and reset iterator
    def reset(self):
        self.interbucket_idx = -1
        for idx in xrange(len(self.bucketed_data)):
            current = self.bucketed_data[idx]
            src, targ = current
            indices = np.array(range(src.shape[0]))
            np.random.shuffle(indices)
            src = src[indices]
            targ = targ[indices]
            self.bucketed_data[idx] = (src, targ)
        shuffle(self.bucket_iterator_indices)

    # iterate over data
    def next(self):
        try:
            if self.switch_bucket:
                self.interbucket_idx += 1
                self.curr_bucket_id = self.bucket_iterator_indices[self.interbucket_idx]
                self.curr_buck = self.bucketed_data[self.curr_bucket_id]
                src_buck_len, src_buck_wid = self.curr_buck[0].shape
                targ_buck_len, targ_buck_wid = self.curr_buck[1].shape                 
                if src_buck_len == 0 or src_buck_wid == 0:
                    raise StopIteration
                if targ_buck_len == 0 or targ_buck_wid == 0:
                    raise StopIteration
                self.curr_chunks = self.chunks(range(src_buck_len), self.batch_size)
                self.switch_bucket = False
            current = self.curr_chunks.next()
            src_ex = ndarray.array(self.curr_buck[0][current])
            targ_ex = ndarray.array(self.curr_buck[1][current])
            
            return DataBatch([src_ex], [targ_ex], pad=0,
                             bucket_key=self.bucket_idx_to_key[self.curr_bucket_id],
                             provide_data=[(self.data_name, src_ex.shape)],
                             provide_label=[(self.label_name, targ_ex.shape)])
                
        except StopIteration as si:
            if self.interbucket_idx == self.num_buckets - 1:
                self.reset()
                self.switch_bucket = True
                raise si
            else:
                self.switch_bucket = True
                return self.next()

    @staticmethod
    def chunks(iterable, batch_size, trim_incomplete_batches=True):
        n = max(1, batch_size)
        end = len(iterable)/n*n if trim_incomplete_batches else len(iterable)
        return (iterable[i:i+n] for i in xrange(0, end, n))
    
    @staticmethod 
    def filter_long_sent(src_sent, targ_sent, max_len):
        result = filter(lambda x: len(x[0]) <= max_len and len(x[1]) <= max_len, zip(src_sent, targ_sent))
        return zip(*result)

    @staticmethod
    def gen_buckets(src_sent, targ_sent, filter_smaller_counts_than=None, max_sent_len=60, min_sent_len=1):
        length_pairs = map(lambda x: (len(x[0]), len(x[1])), zip(src_sent, targ_sent))
        counts = list(Counter(length_pairs).items())
        c_sorted = sorted(counts, key=operator.itemgetter(0, 1))
        buckets = [i[0] for i in c_sorted if i[1] >= filter_smaller_counts_than and 
                   (max_sent_len is None or i[0][0] <= max_sent_len) and
                   (max_sent_len is None or i[0][1] <= max_sent_len) and
                   (min_sent_len is None or i[0][0] >= min_sent_len) and
                   (min_sent_len is None or i[0][1] >= min_sent_len)]
        return buckets

In [8]:
src_sent = dataset.src_sent
targ_sent = dataset.targ_sent

sent_len = lambda x: map(lambda y: len(y), x)
max_len = lambda x: max(sent_len(x))
min_len = lambda x: min(sent_len(x))

min_len = min(min(sent_len(src_sent)), min(sent_len(targ_sent)))

# min_len = min
max_len = 65
increment = 5

all_pairs = [(i, j) for i in xrange(
        min_len,max_len+increment,increment
    ) for j in xrange(
        min_len,max_len+increment,increment
    )]

In [9]:
i2 = Seq2SeqIter(dataset, buckets=all_pairs)

In [10]:
i2.bucketed_data

[(array([], shape=(105, 0), dtype=int32),
  array([[   30,     0,     0,     0,     0],
         [   30,     0,     0,     0,     0],
         [   30,     0,     0,     0,     0],
         [   30,     0,     0,     0,     0],
         [   30,     0,     0,     0,     0],
         [   30,     0,     0,     0,     0],
         [   30,     0,     0,     0,     0],
         [   30,     0,     0,     0,     0],
         [   30,     0,     0,     0,     0],
         [   30,     0,     0,     0,     0],
         [   30,     0,     0,     0,     0],
         [   30,     0,     0,     0,     0],
         [   30,     0,     0,     0,     0],
         [   30,     0,     0,     0,     0],
         [   30,     0,     0,     0,     0],
         [   30,     0,     0,     0,     0],
         [   30,     0,     0,     0,     0],
         [ 3116,    30,     0,     0,     0],
         [  194,    30,     0,     0,     0],
         [ 6596,    30,     0,     0,     0],
         [   30,     0,     0,     0, 

In [11]:
narr = np.random.randn(5, 5)
print(narr)
marr = mx.nd.array(narr).asnumpy()
print(marr)

[[-0.99800026  1.10641942  1.10715411  0.8684042   1.00957505]
 [-1.38617105  0.32404654  0.48824554 -1.19913472  0.59975803]
 [ 0.07791083 -0.67982819 -0.18052017 -1.31626982  1.90677966]
 [ 1.53364845  0.9867157   1.45865342  1.19948852 -2.78914996]
 [ 0.88572197 -0.86513952 -1.66143656 -0.62811988 -1.33853   ]]
[[-0.99800026  1.10641944  1.10715413  0.86840421  1.00957501]
 [-1.3861711   0.32404655  0.48824555 -1.19913471  0.59975803]
 [ 0.07791083 -0.67982817 -0.18052016 -1.31626987  1.90677965]
 [ 1.53364849  0.98671567  1.45865345  1.19948852 -2.78915   ]
 [ 0.88572198 -0.86513954 -1.66143656 -0.62811989 -1.33852994]]


In [ ]:
def print_text(iterator, vocab, text, max_examples=10):
    for i in range(min(max_examples, len(src))):
        x = text[i]
        s = []
        for j in range(len(x)):
            s.append(vocab[x[j]])
        print(" ".join(s))

In [ ]:
print_text(i2, i2.inv_src_vocab, src)

In [ ]:
mx.nd.array(np.array([1,2,3]))

In [12]:
src

NameError: name 'src' is not defined

In [13]:
dir(i2)

['TwoDBisect',
 '__class__',
 '__delattr__',
 '__dict__',
 '__doc__',
 '__format__',
 '__getattribute__',
 '__hash__',
 '__init__',
 '__iter__',
 '__module__',
 '__new__',
 '__next__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'batch_size',
 'bisect',
 'bucket_idx_to_key',
 'bucket_iterator_indices',
 'bucket_key_to_idx',
 'bucketed_data',
 'bucketize',
 'buckets',
 'chunks',
 'curr_buck',
 'curr_bucket_id',
 'curr_chunks',
 'current_bucket_index',
 'current_bucket_key',
 'data_name',
 'dtype',
 'filter_long_sent',
 'gen_buckets',
 'getdata',
 'getindex',
 'getlabel',
 'getpad',
 'interbucket_idx',
 'inv_src_vocab',
 'inv_targ_vocab',
 'iter_next',
 'label_name',
 'layout',
 'max_sent_len',
 'next',
 'num_buckets',
 'pad_id',
 'reset',
 'src_sent',
 'src_vocab',
 'switch_bucket',
 'targ_sent',
 'targ_vocab']

In [ ]:
print_text(i2, i2.inv_targ_vocab, targ)